In [1]:
import pandas as pd
import json

from utils.plotly_theme import *

In [2]:
import json

with open("data/input/equipment_data.json", "r") as f:
    data = json.load(f)
    eq_types = [item["eq_type"] for item in data["equipment"]]

equipment_types = list(set(eq_types))
print(equipment_types)

['resistance_heater', 'hr_heat_pump', 'heat_pump', 'chiller', 'boiler']


In [3]:
# test interpolation

from utils.interp import interp_vector

fp = [3.815, 3.955, 4.133, 3.74] # cop
xp = [1465000, 1098750, 732500, 366250] # cap
x = [1465200]
print(interp_vector(xp, fp, x))

[3.74]


In [4]:
# Example usage of the decarb-tool

from src.metadata import Metadata, EmissionScenario
from utils.inputs import get_city_and_zone

# Load location data
location_data = pd.read_csv("data/input/locations.csv")
zip_code = "11229" # Brooklyn, NY -> will be replaced by user input from UI (search by zip-code or city name)
locations = get_city_and_zone(zip_code, location_data)

# Based on zip-code, SET location, climate_zone, time_zone
location = locations.city
ashrae_climate_zone = locations.ashrae_climate_zone
time_zone = locations.time_zone

# Set building type, building vintage, load type and equipment via equipment scenario -> eventually replaced by user input from UI
building_type = "Hospital"
vintage = 2004
load_type = "load_simulated"
equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"] # list of equipment scenarios to compare

settings = Metadata.create(
    location=location,
    ashrae_climate_zone=ashrae_climate_zone,
    time_zone=time_zone,
    building_type=building_type,
    vintage=vintage,
    load_type=load_type,
    equipment_scenarios=equipment_scenarios,
)

scenario = settings.get_emission_scenario("em_scenario_a")
print(scenario.year)

2025


In [5]:
settings.model_dump()

{'location': 'New York',
 'building_type': 'Hospital',
 'vintage': 2004,
 'load_type': 'load_simulated',
 'ashrae_climate_zone': '4A',
 'equipment_scenarios': ['eq_scenario_1',
  'eq_scenario_2',
  'eq_scenario_3',
  'eq_scenario_4',
  'eq_scenario_5'],
 'emission_settings': [{'em_scen_id': 'em_scenario_a',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Combustion only',
   'shortrun_weighting': 1.0,
   'annual_refrig_leakage': 0.01,
   'annual_ng_leakage': 0.005,
   'year': 2025},
  {'em_scen_id': 'em_scenario_b',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_type': 'Combustion only',
   'shortrun_weighting': 0.5,
   'annual_refrig_leakage': 0.01,
   'annual_ng_leakage': 0.005,
   'year': 2035},
  {'em_scen_id': 'em_scenario_c',
   'grid_scenario': 'MidCase',
   'gea_grid_region': 'CAISO',
   'time_zone': 'America/Los_Angeles',
   'emission_typ

In [6]:
from src.equipment import load_library
equipment_library = load_library("data/input/equipment_data.json").model_dump()

equipment_library

{'equipment': [{'eq_id': 'hr01',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water',
   'eq_manufacturer': None,
   'model': 'GenericHR_WWHP',
   'fuel': 'electricity',
   'capacity_W': None,
   'performance': {'cop_curve': None,
    'cap_curve': None,
    'plr_curve': {'capacity_W': [146500.0, 109875.0, 73250.0, 36625.0],
     'cop': [3.815, 3.955, 4.133, 3.74]},
    'efficiency': None,
    'constraints': None},
   'emissions': None},
  {'eq_id': 'hr02',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water_scroll',
   'eq_manufacturer': None,
   'model': 'IMC-WWHP-30Ton-120HHWST',
   'fuel': 'electricity',
   'capacity_W': None,
   'performance': {'cop_curve': None,
    'cap_curve': None,
    'plr_curve': {'capacity_W': [105500.0, 50000.0], 'cop': [3.68, 4.2]},
    'efficiency': None,
    'constraints': None},
   'emissions': None},
  {'eq_id': 'hr03',
   'eq_type': 'hr_heat_pump',
   'eq_subtype': 'water_to_water_scroll',
   'eq_manufacturer': None,
   'model':

In [7]:
equipment_data = pd.json_normalize(equipment_library["equipment"])
equipment_data.head(50)

,eq_id,eq_type,eq_subtype,eq_manufacturer,model,fuel,capacity_W,emissions,performance.cop_curve,performance.cap_curve,performance.plr_curve.capacity_W,performance.plr_curve.cop,performance.efficiency,performance.constraints,performance.cop_curve.t_out_C,performance.cop_curve.cop,performance.cap_curve.t_out_C,performance.cap_curve.capacity_W,performance.plr_curve,emissions.co2_kg_per_mwh
0,hr01,hr_heat_pump,water_to_water,None,GenericHR_WWHP,electricity,NaN,NaN,NaN,NaN,"[146500.0, 109875.0, 73250.0, 36625.0]","[3.815, 3.955, 4.133, 3.74]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
1,hr02,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[105500.0, 50000.0]","[3.68, 4.2]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
2,hr03,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-30Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[105500.0, 50000.0]","[2.68, 3.3]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
3,hr04,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[211000.0, 100000.0]","[3.68, 4.2]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
4,hr05,hr_heat_pump,water_to_water_scroll,None,IMC-WWHP-60Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[211000.0, 100000.0]","[2.68, 3.3]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
5,hr06,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[879250.0, 700000.0, 500000.0, 300000.0]","[3.68, 3.9, 4.3, 4.5]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
6,hr07,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-250Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[879250.0, 700000.0, 500000.0, 300000.0]","[2.68, 2.9, 3.3, 3.5]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
7,hr08,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-120HHWST,electricity,NaN,NaN,NaN,NaN,"[1758500.0, 1200000.0, 800000.0, 600000.0]","[3.96, 4.2, 4.6, 4.8]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
8,hr09,hr_heat_pump,water_to_water_centrifugal,None,IMC-WWHP-500Ton-140HHWST,electricity,NaN,NaN,NaN,NaN,"[1758500.0, 1200000.0, 800000.0, 600000.0]","[2.97, 3.2, 3.6, 3.8]",NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
9,hp01,heat_pump,air_to_water,None,GenericAWHP,electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"[-10.0, 0.0, 10.0, 20.0, 30.0]","[2.0, 2.5, 3.2, 3.8, 4.2]","[-10.0, 0.0, 10.0, 20.0, 30.0]","[100000.0, 120000.0, 140000.0, 160000.0, 18000...",NaN,NaN


In [8]:
settings.add_emission_scenario(
    EmissionScenario(
        em_scen_id="new_007",
        grid_scenario="MidCase",
        gea_grid_region="NYISO",
        time_zone="America/New_York",
        emission_type="Combustion only",
        annual_ng_leakage=2.0,
        annual_refrig_leakage=5.0,
        shortrun_weighting=1.0,
        year=2030,
    )
)

print(settings["new_007"].year)
print(settings["new_007"].grid_scenario)

x = settings.__getitem__("new_007")
print(x.gea_grid_region)


2030
MidCase
NYISO


In [9]:

from pprint import pprint


pprint(settings.dict())

{'ashrae_climate_zone': '4A',
 'building_type': 'Hospital',
 'emission_settings': [{'annual_ng_leakage': 0.005,
                        'annual_refrig_leakage': 0.01,
                        'em_scen_id': 'em_scenario_a',
                        'emission_type': 'Combustion only',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 1.0,
                        'time_zone': 'America/Los_Angeles',
                        'year': 2025},
                       {'annual_ng_leakage': 0.005,
                        'annual_refrig_leakage': 0.01,
                        'em_scen_id': 'em_scenario_b',
                        'emission_type': 'Combustion only',
                        'gea_grid_region': 'CAISO',
                        'grid_scenario': 'MidCase',
                        'shortrun_weighting': 0.5,
                        'time_zone': 'America/Los_Angeles',
                        'ye

/var/folders/x4/3lxftn1s4zgf32y4j4tqx4kh0000gn/T/ipykernel_87011/996045595.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(settings.dict())


In [10]:
# Get datasets based on input (location, load_type) saved in settings object
from src.loads import get_load_data
from src.emissions import get_emissions_data

# get load data
load_data = get_load_data(settings)

# get emission data
emissions_data = get_emissions_data(settings["em_scenario_a"])

# inspect load data
load_data.df.head(30)

/Users/toby/Desktop/Repositories/decarb-tool/src/loads.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=False)


,t_out_C,heating_W,cooling_W
timestamp,,,
2001-01-01 01:00:00,-4.400000,609230.1302,127902.411800
2001-01-01 02:00:00,-4.400000,606900.3412,174577.649100
2001-01-01 03:00:00,-4.108333,604168.5347,186763.682700
2001-01-01 04:00:00,-3.550000,600461.6520,199730.652500
2001-01-01 05:00:00,-3.300000,604702.9970,205553.300800
2001-01-01 06:00:00,-3.650000,607617.8109,200142.297400
2001-01-01 07:00:00,-4.191667,611215.7286,191604.123600
2001-01-01 08:00:00,-3.758333,607994.1257,113827.739800
2001-01-01 09:00:00,-3.008333,546663.1432,109587.775600


In [11]:
# inspect emissions data
emissions_data.df.head(30)

,em_scen_id,emission_scenario,gea_grid_region,time_zone,emission_type,year,lrmer_co2e_c,lrmer_co2e_p,srmer_co2e_c,srmer_co2e_p
timestamp,,,,,,,,,,
2025-01-01 00:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,408.0,137.7,561.2,173.2
2025-01-01 01:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,431.1,142.3,547.6,163.8
2025-01-01 02:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,429.1,147.9,461.2,162.2
2025-01-01 03:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,413.6,149.6,480.7,167.0
2025-01-01 04:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,417.8,145.3,480.9,179.3
2025-01-01 05:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,396.4,137.6,533.5,174.0
2025-01-01 06:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,364.3,126.7,546.2,177.9
2025-01-01 07:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,269.0,91.7,656.7,154.8
2025-01-01 08:00:00,em_scenario_a,MidCase,CAISO,America/Los_Angeles,Combustion only,2025,139.6,46.7,554.9,158.2


In [12]:
# Test-load equipment library (not changing the actual sim settings)

from src.equipment import load_library

library = load_library("data/input/equipment_data.json")

hp_a = library.get_equipment("hp01")
print(hp_a)

scenario_a = library.get_scenario("eq_scenario_1")
print(scenario_a)

eq_id='hp01' eq_type='heat_pump' eq_subtype='air_to_water' eq_manufacturer=None model='GenericAWHP' fuel='electricity' capacity_W=None performance=Performance(cop_curve=COPCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], cop=[2.0, 2.5, 3.2, 3.8, 4.2]), cap_curve=CapCurve(t_out_C=[-10.0, 0.0, 10.0, 20.0, 30.0], capacity_W=[100000.0, 120000.0, 140000.0, 160000.0, 180000.0]), plr_curve=None, efficiency=None, constraints=None) emissions=None
eq_scen_id='eq_scenario_1' eq_scen_name='Baseline System' hr_wwhp=None awhp_h='hp01' awhp_c='hp01' awhp_sizing=1.0 boiler='bo01' chiller='ch01' resistance_heater='res01'


In [13]:
settings.equipment_scenarios

['eq_scenario_1',
 'eq_scenario_2',
 'eq_scenario_3',
 'eq_scenario_4',
 'eq_scenario_5']

In [14]:
from src.energy import loads_to_site_energy

site_energy = loads_to_site_energy(load_data, library, settings.equipment_scenarios, detail=True)

site_energy.head(30)


,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,awhp_cop_c,awhp_chw_W,elec_awhp_c_Wh,chiller_cop,chiller_chw_W,elec_chiller_Wh,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-01 01:00:00,-4.40,609230.13,127902.41,101719.46,622537.66,609230.13,127902.41,NaN,NaN,NaN,...,2.28,111200.00,48771.93,4.0,16702.41,4175.60,eq_scenario_1,NaN,NaN,NaN
2001-01-01 02:00:00,-4.40,606900.34,174577.65,113388.27,619625.43,606900.34,174577.65,NaN,NaN,NaN,...,2.28,111200.00,48771.93,4.0,63377.65,15844.41,eq_scenario_1,NaN,NaN,NaN
2001-01-01 03:00:00,-4.11,604168.53,186763.68,116177.45,615481.50,604168.53,186763.68,NaN,NaN,NaN,...,2.29,111783.33,48716.18,4.0,74980.35,18745.09,eq_scenario_1,NaN,NaN,NaN
2001-01-01 04:00:00,-3.55,600461.65,199730.65,118930.48,609452.06,600461.65,199730.65,NaN,NaN,NaN,...,2.32,112900.00,48611.41,4.0,86830.65,21707.66,eq_scenario_1,NaN,NaN,NaN
2001-01-01 05:00:00,-3.30,604703.00,205553.30,120168.95,614128.75,604703.00,205553.30,NaN,NaN,NaN,...,2.34,113400.00,48565.31,4.0,92153.30,23038.33,eq_scenario_1,NaN,NaN,NaN
2001-01-01 06:00:00,-3.65,607617.81,200142.30,119120.55,618647.26,607617.81,200142.30,NaN,NaN,NaN,...,2.32,112700.00,48629.99,4.0,87442.30,21860.57,eq_scenario_1,NaN,NaN,NaN
2001-01-01 07:00:00,-4.19,611215.73,191604.12,117460.94,624498.83,611215.73,191604.12,NaN,NaN,NaN,...,2.29,111616.67,48732.04,4.0,79987.46,19996.86,eq_scenario_1,NaN,NaN,NaN
2001-01-01 08:00:00,-3.76,607994.13,113827.74,97636.52,619388.49,607994.13,113827.74,NaN,NaN,NaN,...,2.31,112483.33,48650.21,4.0,1344.41,336.10,eq_scenario_1,NaN,NaN,NaN
2001-01-01 09:00:00,-3.01,546663.14,109587.78,95153.51,540849.76,546663.14,109587.78,NaN,NaN,NaN,...,2.35,109587.78,46641.37,4.0,0.00,0.00,eq_scenario_1,NaN,NaN,NaN


In [15]:
settings.list_emission_scenarios()

['em_scenario_a', 'em_scenario_b', 'em_scenario_c', 'new_007']

In [16]:
from src.energy import site_to_source

source_energy = site_to_source(site_energy, metadata=settings)

source_energy.head(30)

,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,elec_emissions_rate,year,elec_emissions,gas_emissions,total_refrig_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,-4.40,609230.13,127902.41,101719.46,622537.66,609230.13,127902.41,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,489.35,2025,49.776418,112.056779,0.0,em_scenario_a
2025-01-01 02:00:00,-4.40,606900.34,174577.65,113388.27,619625.43,606900.34,174577.65,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,445.15,2025,50.474788,111.532577,0.0,em_scenario_a
2025-01-01 03:00:00,-4.11,604168.53,186763.68,116177.45,615481.50,604168.53,186763.68,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,447.15,2025,51.948747,110.786670,0.0,em_scenario_a
2025-01-01 04:00:00,-3.55,600461.65,199730.65,118930.48,609452.06,600461.65,199730.65,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,449.35,2025,53.441411,109.701371,0.0,em_scenario_a
2025-01-01 05:00:00,-3.30,604703.00,205553.30,120168.95,614128.75,604703.00,205553.30,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,464.95,2025,55.872553,110.543175,0.0,em_scenario_a
2025-01-01 06:00:00,-3.65,607617.81,200142.30,119120.55,618647.26,607617.81,200142.30,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,455.25,2025,54.229630,111.356507,0.0,em_scenario_a
2025-01-01 07:00:00,-4.19,611215.73,191604.12,117460.94,624498.83,611215.73,191604.12,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,462.85,2025,54.366796,112.409789,0.0,em_scenario_a
2025-01-01 08:00:00,-3.76,607994.13,113827.74,97636.52,619388.49,607994.13,113827.74,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,347.25,2025,33.904282,111.489928,0.0,em_scenario_a
2025-01-01 09:00:00,-3.01,546663.14,109587.78,95153.51,540849.76,546663.14,109587.78,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,292.30,2025,27.813371,97.352957,0.0,em_scenario_a


In [17]:
df = source_energy.copy()

df

,t_out_C,heating_W,cooling_W,elec_Wh,gas_Wh,hhw_W,chw_W,hr_hhw_W,hr_chw_W,hr_cop_h,...,scenario_id,max_cap_h_hr_W,min_cap_h_hr_W,simult_h_hr_W,elec_emissions_rate,year,elec_emissions,gas_emissions,total_refrig_emissions,em_scen_id
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-01-01 01:00:00,-4.40,609230.13,127902.41,101719.46,622537.66,609230.13,127902.41,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,489.35,2025,49.776418,112.056779,0.0,em_scenario_a
2025-01-01 02:00:00,-4.40,606900.34,174577.65,113388.27,619625.43,606900.34,174577.65,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,445.15,2025,50.474788,111.532577,0.0,em_scenario_a
2025-01-01 03:00:00,-4.11,604168.53,186763.68,116177.45,615481.50,604168.53,186763.68,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,447.15,2025,51.948747,110.786670,0.0,em_scenario_a
2025-01-01 04:00:00,-3.55,600461.65,199730.65,118930.48,609452.06,600461.65,199730.65,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,449.35,2025,53.441411,109.701371,0.0,em_scenario_a
2025-01-01 05:00:00,-3.30,604703.00,205553.30,120168.95,614128.75,604703.00,205553.30,NaN,NaN,NaN,...,eq_scenario_1,NaN,NaN,NaN,464.95,2025,55.872553,110.543175,0.0,em_scenario_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 20:00:00,-5.89,470817.61,73010.85,321937.16,0.00,470817.61,73010.85,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,425.45,2030,136.968165,0.000000,0.0,new_007
2030-12-31 21:00:00,-6.45,488341.50,59508.77,333911.71,0.00,488341.50,59508.77,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,369.20,2030,123.280203,0.000000,0.0,new_007
2030-12-31 22:00:00,-6.99,509902.48,46972.96,349307.55,0.00,509902.48,46972.96,NaN,NaN,NaN,...,eq_scenario_5,NaN,NaN,NaN,359.85,2030,125.698322,0.000000,0.0,new_007


In [18]:
#! Update for mutliple scenarios and years

from src.visuals import plot_energy_breakdown

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2"]
emission_scenarios = ["em_scenario_a"]

plot_energy_breakdown(df, equipment_scenarios, emission_scenarios)


In [19]:

from src.visuals import plot_energy_and_emissions

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"]
emission_scenarios = ["em_scenario_a"]

plot_energy_and_emissions(df, equipment_scenarios, emission_scenarios)


In [20]:
#! Update for mutliple scenarios

from src.visuals import plot_emissions_heatmap

df = source_energy.copy()

year = 2040

plot_emissions_heatmap(df, year)


In [21]:
from src.visuals import plot_meter_timeseries

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1"]
emission_scenarios = ["em_scenario_b"]

plot_meter_timeseries(df, equipment_scenarios=equipment_scenarios, emission_scenarios=emission_scenarios, stacked=True, include_gas=True)

In [22]:
from src.visuals import plot_total_emissions_bar

df = source_energy.copy()

equipment_scenarios = ["eq_scenario_1", "eq_scenario_2", "eq_scenario_3", "eq_scenario_4", "eq_scenario_5"]
emission_scenarios = ["em_scenario_a", "em_scenario_b"]

plot_total_emissions_bar(df, equipment_scenarios=equipment_scenarios, emission_scenarios=emission_scenarios, unit_mode="IP")